# Train your first 🐸 TTS model 💫

### 👋 Hello and welcome to Coqui (🐸) TTS

The goal of this notebook is to show you a **typical workflow** for **training** and **testing** a TTS model with 🐸.

Let's train a very small model on a very small amount of data so we can iterate quickly.

In this notebook, we will:

1. Download data and format it for 🐸 TTS.
2. Configure the training and testing runs.
3. Train a new model.
4. Test the model and display its performance.

So, let's jump right in!


In [1]:
## Install Coqui TTS
! pip install -U pip
! pip install TTS

## ✅ Data Preparation

### **First things first**: we need some data.

We're training a Text-to-Speech model, so we need some _text_ and we need some _speech_. Specificially, we want _transcribed speech_. The speech must be divided into audio clips and each clip needs transcription. More details about data requirements such as recording characteristics, background noise abd vocabulary coverage can be found in the [🐸TTS documentation](https://tts.readthedocs.io/en/latest/formatting_your_dataset.html).

If you have a single audio file and you need to **split** it into clips. It is also important to use a lossless audio file format to prevent compression artifacts. We recommend using **wav** file format.

The data format we will be adopting for this tutorial is taken from the widely-used  **LJSpeech** dataset, where **waves** are collected under a folder:

<span style="color:purple;font-size:15px">
/wavs<br /> 
 &emsp;| - audio1.wav<br /> 
 &emsp;| - audio2.wav<br /> 
 &emsp;| - audio3.wav<br /> 
  ...<br /> 
</span>

and a **metadata.csv** file will have the audio file name in parallel to the transcript, delimited by `|`: 
 
<span style="color:purple;font-size:15px">
# metadata.csv <br /> 
audio1|This is my sentence. <br /> 
audio2|This is maybe my sentence. <br /> 
audio3|This is certainly my sentence. <br /> 
audio4|Let this be your sentence. <br /> 
...
</span>

In the end, we should have the following **folder structure**:

<span style="color:purple;font-size:15px">
/MyTTSDataset <br /> 
&emsp;| <br /> 
&emsp;| -> metadata.csv<br /> 
&emsp;| -> /wavs<br /> 
&emsp;&emsp;| -> audio1.wav<br /> 
&emsp;&emsp;| -> audio2.wav<br /> 
&emsp;&emsp;| ...<br /> 
</span>

🐸TTS already provides tooling for the _LJSpeech_. if you use the same format, you can start training your models right away. <br /> 

After you collect and format your dataset, you need to check two things. Whether you need a **_formatter_** and a **_text_cleaner_**. <br /> The **_formatter_** loads the text file (created above) as a list and the **_text_cleaner_** performs a sequence of text normalization operations that converts the raw text into the spoken representation (e.g. converting numbers to text, acronyms, and symbols to the spoken format).

If you use a different dataset format then the LJSpeech or the other public datasets that 🐸TTS supports, then you need to write your own **_formatter_** and  **_text_cleaner_**.

## ⏳️ Loading your dataset
Load one of the dataset supported by 🐸TTS.

We will start by defining dataset config and setting LJSpeech as our target dataset and define its path.


In [2]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)
    

In [4]:
# Download and extract LJSpeech dataset.

!wget -O $output_path/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 --no-check-certificate
!tar -xf $output_path/LJSpeech-1.1.tar.bz2 -C $output_path

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)


2499500K .......... .......... .......... .......... .......... 93% 10.3M 25s
2499550K .......... .......... .......... .......... .......... 93% 6.67M 25s
2499600K .......... .......... .......... .......... .......... 93% 9.53M 25s
2499650K .......... .......... .......... .......... .......... 93% 11.7M 25s
2499700K .......... .......... .......... .......... .......... 93% 6.00M 25s
2499750K .......... .......... .......... .......... .......... 93% 12.0M 25s
2499800K .......... .......... .......... .......... .......... 93% 11.1M 25s
2499850K .......... .......... .......... .......... .......... 93% 9.71M 25s
2499900K .......... .......... .......... .......... .

In [5]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "LJSpeech-1.1/")
)

## ✅ Train a new model

Let's kick off a training run 🚀🚀🚀.

Deciding on the model architecture you'd want to use is based on your needs and available resources. Each model architecture has it's pros and cons that define the run-time efficiency and the voice quality.
We have many recipes under `TTS/recipes/` that provide a good starting point. For this tutorial, we will be using `GlowTTS`.

We will begin by initializing the model training configuration.

In [6]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

Next we will initialize the audio processor which is used for feature extraction and audio I/O.

In [7]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)
# Modify sample rate if for a custom audio dataset:
# ap.sample_rate = 22050


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


Next we will initialize the tokenizer which is used to convert text to sequences of token IDs.  If characters are not defined in the config, default characters are passed to the config.

In [8]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

Next we will load data samples. Each sample is a list of ```[text, audio_file_path, speaker_name]```. You can define your custom sample loader returning the list of samples.

In [9]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 13100 files in C:\Users\iambl\GitHub\TTS\notebooks\tts_train_dir\LJSpeech-1.1


Now we're ready to initialize the model.

Models take a config object and a speaker manager as input. Config defines the details of the model like the number of layers, the size of the embedding, etc. Speaker manager is used by multi-speaker models.

In [10]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

C:\Users\iambl\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\compliance\kaldi.py:22: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  EPSILON = torch.tensor(torch.finfo(torch.float).eps)


Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training, distributed training, etc.

In [11]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Num. of CPUs: 16
 | > Num. of Torch Threads: 8
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Start Tensorboard: tensorboard --logdir=tts_train_dir\run-February-15-2023_12+37AM-661725b9

 > Model has 28610257 parameters


### AND... 3,2,1... START TRAINING 🚀🚀🚀

In [ ]:
trainer.fit()


 > EPOCH: 0/100
 --> tts_train_dir\run-February-15-2023_12+37AM-661725b9


[*] Pre-computing phonemes...


  1%|▌                                                                                                   | 75/12969 [00:05<17:00, 12.63it/s]

ðə kəmˈɪʃən bᵻlˈiːvz ðætðə fˈækts ʌvðɪ ɐsˌæsᵻnˈeɪʃən ʌv pɹˈɛzɪdənt kˈɛnədi pˈɔɪnt tə sˈɜːʔn̩ mˈɛʒɚz wˈɪtʃ,
 [!] Character '̩' not found in the vocabulary. Discarding it.


 16%|███████████████▌                                                                                  | 2056/12969 [02:50<18:42,  9.72it/s]

ˈɪntʊ ðə “kɹˈeɪɾɚ” dˈʌɡ ˈaʊt ɪnðə mˈɪdəl, pˈoːɹ ðə spˈʌndʒ, wˈɔːɹm wˈɔːɾɚ, ðə məlˈæsᵻz, ænd sˈoʊdə dɪsˈɑːlvd ɪn hˈɑːt wˈɔːɾɚ.
 [!] Character '“' not found in the vocabulary. Discarding it.
ˈɪntʊ ðə “kɹˈeɪɾɚ” dˈʌɡ ˈaʊt ɪnðə mˈɪdəl, pˈoːɹ ðə spˈʌndʒ, wˈɔːɹm wˈɔːɾɚ, ðə məlˈæsᵻz, ænd sˈoʊdə dɪsˈɑːlvd ɪn hˈɑːt wˈɔːɾɚ.
 [!] Character '”' not found in the vocabulary. Discarding it.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12969/12969 [17:54<00:00, 12.07it/s]




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: espeak
	| > 3 not found characters:
	| > ̩
	| > “
	| > ”
| > Number of instances : 12969



 > TRAINING (2023-02-15 00:55:57) 


 | > Preprocessing samples
 | > Max text length: 188
 | > Min text length: 13
 | > Avg text length: 100.90014650319993
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 24499.0
 | > Avg audio length: 144984.29755570978
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


#### 🚀 Run the Tensorboard. 🚀
On the notebook and Tensorboard, you can monitor the progress of your model. Also Tensorboard provides certain figures and sample outputs.

In [ ]:
!pip install tensorboard
!tensorboard --logdir=tts_train_dir

## ✅ Test the model

We made it! 🙌

Let's kick off the testing run, which displays performance metrics.

We're committing the cardinal sin of ML 😈 (aka - testing on our training data) so you don't want to deploy this model into production. In this notebook we're focusing on the workflow itself, so it's forgivable 😇

You can see from the test output that our tiny model has overfit to the data, and basically memorized this one sentence.

When you start training your own models, make sure your testing data doesn't include your training data 😅

Let's get the latest saved checkpoint. 

In [ ]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [ ]:
 !tts --text "Text for TTS" \
      --model_path $test_ckpt \
      --config_path $test_config \
      --out_path out.wav

## 📣 Listen to the synthesized wave 📣

In [ ]:
import IPython
IPython.display.Audio("out.wav")

## 🎉 Congratulations! 🎉 You now have trained your first TTS model! 
Follow up with the next tutorials to learn more advanced material.